# Service Integration for AG News Text Classification

## Overview

This notebook demonstrates comprehensive service integration patterns following methodologies from:
- Newman (2015): "Building Microservices: Designing Fine-Grained Systems"
- Richardson (2018): "Microservices Patterns: With Examples in Java"
- Kleppmann (2017): "Designing Data-Intensive Applications"

### Tutorial Objectives
1. Integrate core classification services
2. Implement service orchestration
3. Configure message queuing systems
4. Set up caching strategies
5. Connect storage services
6. Enable monitoring and alerting
7. Deploy notification systems
8. Manage service pipelines

Author: Võ Hải Dũng  
Email: vohaidung.work@gmail.com  
Date: 2025

## 1. Environment Setup

In [ ]:
# Standard library imports
import sys
import os
import json
import asyncio
import time
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Any, Union
from dataclasses import dataclass, asdict, field
from enum import Enum
import warnings

# Service and infrastructure imports
import redis
import celery
from kafka import KafkaProducer, KafkaConsumer
import boto3
from prometheus_client import Counter, Histogram, Gauge

# Data manipulation
import numpy as np
import pandas as pd
import torch

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

# Project imports
PROJECT_ROOT = Path("../..").resolve()
sys.path.insert(0, str(PROJECT_ROOT))

from src.services.base_service import BaseService
from src.services.service_registry import ServiceRegistry
from src.services.core.prediction_service import PredictionService
from src.services.core.training_service import TrainingService
from src.services.core.data_service import DataService
from src.services.core.model_management_service import ModelManagementService
from src.services.orchestration.workflow_orchestrator import WorkflowOrchestrator
from src.services.orchestration.pipeline_manager import PipelineManager
from src.services.orchestration.job_scheduler import JobScheduler
from src.services.orchestration.state_manager import StateManager
from src.services.caching.cache_service import CacheService
from src.services.caching.redis_cache import RedisCache
from src.services.queue.task_queue import TaskQueue
from src.services.queue.message_broker import MessageBroker
from src.services.monitoring.metrics_service import MetricsService
from src.services.monitoring.health_service import HealthService
from src.services.notification.notification_service import NotificationService
from src.services.storage.storage_service import StorageService
from src.utils.service_utils import (
    create_service_client,
    handle_service_error,
    service_health_check
)
from src.utils.logging_config import setup_logging
from configs.config_loader import ConfigLoader
from configs.constants import AG_NEWS_CLASSES

# Setup
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
logger = setup_logging('service_integration_tutorial')

# Service Configuration
SERVICE_CONFIG = {
    'redis_host': 'localhost',
    'redis_port': 6379,
    'kafka_broker': 'localhost:9092',
    'celery_broker': 'redis://localhost:6379/0',
    'prometheus_port': 9090,
    'service_timeout': 30,
    'max_retries': 3
}

print("Service Integration Tutorial")
print("="*50)
print(f"Project Root: {PROJECT_ROOT}")
print(f"Redis: {SERVICE_CONFIG['redis_host']}:{SERVICE_CONFIG['redis_port']}")
print(f"Kafka Broker: {SERVICE_CONFIG['kafka_broker']}")

## 2. Service Registry and Discovery

In [ ]:
@dataclass
class ServiceInfo:
    """
    Service information for registry.
    
    Following service discovery patterns from:
        Burns (2018): "Designing Distributed Systems"
    """
    name: str
    version: str
    host: str
    port: int
    protocol: str
    status: str = 'unknown'
    health_endpoint: str = '/health'
    metadata: Dict[str, Any] = field(default_factory=dict)


class ServiceDiscovery:
    """
    Service discovery and registration.
    
    Following patterns from:
        HashiCorp Consul documentation
    """
    
    def __init__(self):
        self.services: Dict[str, ServiceInfo] = {}
        self.registry = ServiceRegistry()
    
    def register_service(self, service_info: ServiceInfo) -> bool:
        """Register a service."""
        try:
            # Check if service is healthy
            if self._health_check(service_info):
                service_info.status = 'healthy'
                self.services[service_info.name] = service_info
                self.registry.register(service_info.name, service_info)
                logger.info(f"Service registered: {service_info.name}")
                return True
            else:
                service_info.status = 'unhealthy'
                logger.warning(f"Service unhealthy: {service_info.name}")
                return False
        except Exception as e:
            logger.error(f"Failed to register service: {e}")
            return False
    
    def discover_service(self, service_name: str) -> Optional[ServiceInfo]:
        """Discover a service by name."""
        if service_name in self.services:
            service = self.services[service_name]
            # Check current health
            if self._health_check(service):
                return service
        return None
    
    def list_services(self, status: Optional[str] = None) -> List[ServiceInfo]:
        """List all registered services."""
        services = list(self.services.values())
        if status:
            services = [s for s in services if s.status == status]
        return services
    
    def _health_check(self, service_info: ServiceInfo) -> bool:
        """Check service health."""
        # Simulate health check
        import random
        return random.random() > 0.1  # 90% healthy


# Initialize service discovery
service_discovery = ServiceDiscovery()

# Register core services
core_services = [
    ServiceInfo(
        name='prediction-service',
        version='1.0.0',
        host='localhost',
        port=8001,
        protocol='http',
        metadata={'model': 'deberta-v3', 'max_batch_size': 32}
    ),
    ServiceInfo(
        name='training-service',
        version='1.0.0',
        host='localhost',
        port=8002,
        protocol='http',
        metadata={'gpu_enabled': True, 'max_parallel_jobs': 4}
    ),
    ServiceInfo(
        name='data-service',
        version='1.0.0',
        host='localhost',
        port=8003,
        protocol='http',
        metadata={'storage_backend': 's3', 'cache_enabled': True}
    ),
    ServiceInfo(
        name='model-management',
        version='1.0.0',
        host='localhost',
        port=8004,
        protocol='http',
        metadata={'model_registry': 'mlflow', 'versioning': True}
    )
]

print("Registering Services:")
print("="*50)

for service in core_services:
    success = service_discovery.register_service(service)
    status = "[OK]" if success else "[FAIL]"
    print(f"{status} {service.name}: {service.host}:{service.port} [{service.status}]")

# List registered services
print("\nRegistered Services:")
registered = service_discovery.list_services()
for service in registered:
    print(f"  - {service.name} (v{service.version}): {service.status}")

## 3. Service Orchestration

In [ ]:
class ServiceOrchestrator:
    """
    Orchestrate multiple services for complex workflows.
    
    Following orchestration patterns from:
        Hohpe & Woolf (2003): "Enterprise Integration Patterns"
    """
    
    def __init__(self, service_discovery: ServiceDiscovery):
        self.service_discovery = service_discovery
        self.workflows = {}
        self.pipeline_manager = PipelineManager()
        self.state_manager = StateManager()
    
    def create_workflow(self, name: str, steps: List[Dict[str, Any]]) -> str:
        """Create a service workflow."""
        workflow_id = f"workflow_{name}_{int(time.time())}"
        
        workflow = {
            'id': workflow_id,
            'name': name,
            'steps': steps,
            'status': 'created',
            'created_at': time.time()
        }
        
        self.workflows[workflow_id] = workflow
        self.state_manager.initialize_state(workflow_id)
        
        return workflow_id
    
    async def execute_workflow(self, workflow_id: str) -> Dict[str, Any]:
        """Execute a workflow asynchronously."""
        workflow = self.workflows.get(workflow_id)
        if not workflow:
            raise ValueError(f"Workflow not found: {workflow_id}")
        
        workflow['status'] = 'running'
        results = {}
        
        try:
            for step_idx, step in enumerate(workflow['steps']):
                service_name = step['service']
                operation = step['operation']
                params = step.get('params', {})
                
                # Discover service
                service_info = self.service_discovery.discover_service(service_name)
                if not service_info:
                    raise Exception(f"Service not available: {service_name}")
                
                # Execute service operation
                result = await self._execute_service_operation(
                    service_info, operation, params
                )
                
                results[f"step_{step_idx}"] = result
                self.state_manager.update_state(
                    workflow_id, f"step_{step_idx}", result
                )
            
            workflow['status'] = 'completed'
            workflow['results'] = results
            
        except Exception as e:
            workflow['status'] = 'failed'
            workflow['error'] = str(e)
            logger.error(f"Workflow failed: {e}")
        
        return workflow
    
    async def _execute_service_operation(
        self,
        service_info: ServiceInfo,
        operation: str,
        params: Dict[str, Any]
    ) -> Dict[str, Any]:
        """Execute a service operation."""
        # Simulate service call
        await asyncio.sleep(0.5)  # Simulate network delay
        
        return {
            'service': service_info.name,
            'operation': operation,
            'status': 'success',
            'result': f"Processed {operation} on {service_info.name}",
            'timestamp': time.time()
        }


# Create orchestrator
orchestrator = ServiceOrchestrator(service_discovery)

# Define classification workflow
classification_workflow = [
    {
        'service': 'data-service',
        'operation': 'load_data',
        'params': {'dataset': 'ag_news', 'split': 'test'}
    },
    {
        'service': 'prediction-service',
        'operation': 'batch_predict',
        'params': {'model_id': 'deberta-v3', 'batch_size': 32}
    },
    {
        'service': 'model-management',
        'operation': 'log_predictions',
        'params': {'experiment_name': 'tutorial'}
    }
]

# Create and execute workflow
print("\nWorkflow Orchestration:")
print("="*50)

workflow_id = orchestrator.create_workflow(
    'classification_pipeline',
    classification_workflow
)

print(f"Created workflow: {workflow_id}")
print("\nExecuting workflow steps:")

# Execute workflow (simulated)
async def run_workflow():
    result = await orchestrator.execute_workflow(workflow_id)
    return result

# Run async workflow
import asyncio
loop = asyncio.get_event_loop()
if loop.is_running():
    # For Jupyter notebooks
    import nest_asyncio
    nest_asyncio.apply()

workflow_result = asyncio.run(run_workflow())

# Display results
for step_name, step_result in workflow_result.get('results', {}).items():
    print(f"\n{step_name}:")
    print(f"  Service: {step_result['service']}")
    print(f"  Operation: {step_result['operation']}")
    print(f"  Status: {step_result['status']}")

print(f"\nWorkflow Status: {workflow_result['status']}")

## 4. Message Queue Integration

In [ ]:
class MessageQueueService:
    """
    Message queue service for asynchronous processing.
    
    Following message queue patterns from:
        Gregor & Hohpe (2003): "Enterprise Integration Patterns"
    """
    
    def __init__(self, broker_url: str):
        self.broker_url = broker_url
        self.queues = {}
        self.consumers = {}
        self.message_broker = MessageBroker(broker_url)
    
    def create_queue(self, queue_name: str, config: Dict[str, Any] = None) -> bool:
        """Create a message queue."""
        try:
            queue_config = config or {
                'durable': True,
                'max_priority': 10,
                'ttl': 3600
            }
            
            self.queues[queue_name] = {
                'name': queue_name,
                'config': queue_config,
                'messages': [],
                'created_at': time.time()
            }
            
            logger.info(f"Queue created: {queue_name}")
            return True
            
        except Exception as e:
            logger.error(f"Failed to create queue: {e}")
            return False
    
    def publish_message(
        self,
        queue_name: str,
        message: Dict[str, Any],
        priority: int = 5
    ) -> str:
        """Publish message to queue."""
        if queue_name not in self.queues:
            raise ValueError(f"Queue not found: {queue_name}")
        
        message_id = f"msg_{int(time.time() * 1000)}"
        
        message_wrapper = {
            'id': message_id,
            'payload': message,
            'priority': priority,
            'timestamp': time.time(),
            'retries': 0
        }
        
        self.queues[queue_name]['messages'].append(message_wrapper)
        logger.info(f"Message published to {queue_name}: {message_id}")
        
        return message_id
    
    def consume_message(self, queue_name: str) -> Optional[Dict[str, Any]]:
        """Consume message from queue."""
        if queue_name not in self.queues:
            raise ValueError(f"Queue not found: {queue_name}")
        
        messages = self.queues[queue_name]['messages']
        if messages:
            # Sort by priority and timestamp
            messages.sort(key=lambda x: (-x['priority'], x['timestamp']))
            message = messages.pop(0)
            logger.info(f"Message consumed from {queue_name}: {message['id']}")
            return message
        
        return None
    
    def register_consumer(
        self,
        queue_name: str,
        handler: callable,
        auto_ack: bool = True
    ):
        """Register a message consumer."""
        if queue_name not in self.queues:
            raise ValueError(f"Queue not found: {queue_name}")
        
        consumer = {
            'queue': queue_name,
            'handler': handler,
            'auto_ack': auto_ack,
            'active': True
        }
        
        consumer_id = f"consumer_{len(self.consumers)}"
        self.consumers[consumer_id] = consumer
        
        return consumer_id


# Create message queue service
mq_service = MessageQueueService(SERVICE_CONFIG['celery_broker'])

# Create queues for different services
queues = [
    'classification_requests',
    'training_jobs',
    'data_processing',
    'model_updates'
]

print("Message Queue Setup:")
print("="*50)

for queue_name in queues:
    success = mq_service.create_queue(queue_name)
    print(f"Created queue: {queue_name} - {'Success' if success else 'Failed'}")

# Publish test messages
print("\nPublishing Messages:")

test_messages = [
    {
        'queue': 'classification_requests',
        'message': {'text': 'Sample news article', 'model_id': 'deberta-v3'},
        'priority': 8
    },
    {
        'queue': 'training_jobs',
        'message': {'dataset': 'ag_news', 'epochs': 3, 'model_type': 'roberta'},
        'priority': 5
    },
    {
        'queue': 'data_processing',
        'message': {'action': 'augment', 'dataset_id': 'train_001'},
        'priority': 3
    }
]

for msg_config in test_messages:
    msg_id = mq_service.publish_message(
        msg_config['queue'],
        msg_config['message'],
        msg_config['priority']
    )
    print(f"  Published to {msg_config['queue']}: {msg_id}")

# Consume messages
print("\nConsuming Messages:")

for queue_name in ['classification_requests', 'training_jobs']:
    message = mq_service.consume_message(queue_name)
    if message:
        print(f"\n{queue_name}:")
        print(f"  ID: {message['id']}")
        print(f"  Priority: {message['priority']}")
        print(f"  Payload: {message['payload']}")

## 5. Caching Service

In [ ]:
class CachingService:
    """
    Distributed caching service.
    
    Following caching strategies from:
        Fitzpatrick (2004): "Distributed Caching with Memcached"
    """
    
    def __init__(self, cache_config: Dict[str, Any]):
        self.config = cache_config
        self.cache_stores = {}
        self.stats = {
            'hits': 0,
            'misses': 0,
            'evictions': 0
        }
        self._initialize_stores()
    
    def _initialize_stores(self):
        """Initialize cache stores."""
        # Create different cache levels
        self.cache_stores['L1'] = {}  # In-memory cache
        self.cache_stores['L2'] = {}  # Redis cache (simulated)
        self.cache_stores['L3'] = {}  # Disk cache (simulated)
    
    def get(
        self,
        key: str,
        cache_level: str = 'L1'
    ) -> Optional[Any]:
        """Get value from cache."""
        if cache_level in self.cache_stores:
            if key in self.cache_stores[cache_level]:
                self.stats['hits'] += 1
                entry = self.cache_stores[cache_level][key]
                
                # Check TTL
                if time.time() < entry['expires_at']:
                    # Update access time
                    entry['last_accessed'] = time.time()
                    entry['access_count'] += 1
                    return entry['value']
                else:
                    # Expired, remove from cache
                    self._evict(key, cache_level)
        
        self.stats['misses'] += 1
        return None
    
    def set(
        self,
        key: str,
        value: Any,
        ttl: int = 3600,
        cache_level: str = 'L1'
    ) -> bool:
        """Set value in cache."""
        if cache_level not in self.cache_stores:
            return False
        
        entry = {
            'value': value,
            'created_at': time.time(),
            'expires_at': time.time() + ttl,
            'last_accessed': time.time(),
            'access_count': 0,
            'size': sys.getsizeof(value)
        }
        
        self.cache_stores[cache_level][key] = entry
        
        # Check cache size and evict if necessary
        self._check_cache_size(cache_level)
        
        return True
    
    def invalidate(self, key: str, cache_level: Optional[str] = None) -> bool:
        """Invalidate cache entry."""
        invalidated = False
        
        if cache_level:
            if cache_level in self.cache_stores and key in self.cache_stores[cache_level]:
                del self.cache_stores[cache_level][key]
                invalidated = True
        else:
            # Invalidate from all levels
            for level in self.cache_stores:
                if key in self.cache_stores[level]:
                    del self.cache_stores[level][key]
                    invalidated = True
        
        return invalidated
    
    def _evict(self, key: str, cache_level: str):
        """Evict entry from cache."""
        if key in self.cache_stores[cache_level]:
            del self.cache_stores[cache_level][key]
            self.stats['evictions'] += 1
    
    def _check_cache_size(self, cache_level: str):
        """Check and manage cache size."""
        max_entries = {'L1': 100, 'L2': 1000, 'L3': 10000}
        
        if len(self.cache_stores[cache_level]) > max_entries.get(cache_level, 100):
            # LRU eviction
            entries = self.cache_stores[cache_level]
            sorted_keys = sorted(
                entries.keys(),
                key=lambda k: entries[k]['last_accessed']
            )
            
            # Evict oldest 10%
            evict_count = len(sorted_keys) // 10
            for key in sorted_keys[:evict_count]:
                self._evict(key, cache_level)
    
    def get_stats(self) -> Dict[str, Any]:
        """Get cache statistics."""
        total_ops = self.stats['hits'] + self.stats['misses']
        hit_rate = self.stats['hits'] / total_ops if total_ops > 0 else 0
        
        return {
            'hits': self.stats['hits'],
            'misses': self.stats['misses'],
            'hit_rate': hit_rate,
            'evictions': self.stats['evictions'],
            'cache_sizes': {
                level: len(store) for level, store in self.cache_stores.items()
            }
        }


# Initialize caching service
cache_config = {
    'redis_host': SERVICE_CONFIG['redis_host'],
    'redis_port': SERVICE_CONFIG['redis_port'],
    'default_ttl': 3600,
    'max_memory': '1GB'
}

cache_service = CachingService(cache_config)

print("Caching Service Demo:")
print("="*50)

# Test caching operations
test_data = [
    ('model_predictions_1', {'predictions': [0, 1, 2, 3], 'confidence': [0.9, 0.8, 0.95, 0.7]}),
    ('dataset_meta', {'name': 'ag_news', 'size': 120000, 'classes': 4}),
    ('feature_cache_1', np.random.randn(100, 768).tolist()),
]

# Set cache entries
print("Setting cache entries:")
for key, value in test_data:
    success = cache_service.set(key, value, ttl=1800)
    print(f"  {key}: {'Cached' if success else 'Failed'}")

# Test cache hits and misses
print("\nTesting cache access:")
test_keys = ['model_predictions_1', 'dataset_meta', 'non_existent_key']

for key in test_keys:
    value = cache_service.get(key)
    status = 'HIT' if value is not None else 'MISS'
    print(f"  {key}: {status}")

# Multi-level caching
print("\nMulti-level caching:")
cache_service.set('hot_data', {'value': 'frequently_accessed'}, cache_level='L1')
cache_service.set('warm_data', {'value': 'occasionally_accessed'}, cache_level='L2')
cache_service.set('cold_data', {'value': 'rarely_accessed'}, cache_level='L3')

# Display cache statistics
stats = cache_service.get_stats()
print("\nCache Statistics:")
print(f"  Hits: {stats['hits']}")
print(f"  Misses: {stats['misses']}")
print(f"  Hit Rate: {stats['hit_rate']:.2%}")
print(f"  Evictions: {stats['evictions']}")
print("\nCache Sizes:")
for level, size in stats['cache_sizes'].items():
    print(f"  {level}: {size} entries")

## 6. Storage Service Integration

In [ ]:
class StorageServiceIntegration:
    """
    Unified storage service for multiple backends.
    
    Following storage patterns from:
        AWS Well-Architected Framework - Storage Pillar
    """
    
    def __init__(self):
        self.backends = {}
        self.default_backend = 'local'
        self._initialize_backends()
    
    def _initialize_backends(self):
        """Initialize storage backends."""
        # Local file storage
        self.backends['local'] = {
            'type': 'filesystem',
            'base_path': PROJECT_ROOT / 'data',
            'available': True
        }
        
        # S3 storage (simulated)
        self.backends['s3'] = {
            'type': 's3',
            'bucket': 'ag-news-models',
            'region': 'us-west-2',
            'available': False  # Would be True if configured
        }
        
        # GCS storage (simulated)
        self.backends['gcs'] = {
            'type': 'gcs',
            'bucket': 'ag-news-storage',
            'project': 'ml-project',
            'available': False  # Would be True if configured
        }
    
    def store(
        self,
        key: str,
        data: Any,
        backend: Optional[str] = None,
        metadata: Optional[Dict[str, Any]] = None
    ) -> Dict[str, Any]:
        """Store data in specified backend."""
        backend = backend or self.default_backend
        
        if backend not in self.backends:
            raise ValueError(f"Unknown backend: {backend}")
        
        if not self.backends[backend]['available']:
            logger.warning(f"Backend {backend} not available, using local")
            backend = 'local'
        
        # Simulate storage operation
        storage_info = {
            'key': key,
            'backend': backend,
            'size': sys.getsizeof(data),
            'timestamp': time.time(),
            'metadata': metadata or {},
            'location': self._get_storage_location(backend, key)
        }
        
        logger.info(f"Stored {key} in {backend}")
        return storage_info
    
    def retrieve(
        self,
        key: str,
        backend: Optional[str] = None
    ) -> Optional[Any]:
        """Retrieve data from storage."""
        backend = backend or self.default_backend
        
        # Simulate retrieval
        logger.info(f"Retrieved {key} from {backend}")
        
        # Return simulated data
        return {'data': f"Retrieved {key}", 'backend': backend}
    
    def list_objects(
        self,
        prefix: str = '',
        backend: Optional[str] = None
    ) -> List[Dict[str, Any]]:
        """List objects in storage."""
        backend = backend or self.default_backend
        
        # Simulate object listing
        objects = [
            {'key': f'{prefix}model_v1.pt', 'size': 524288000, 'modified': time.time() - 86400},
            {'key': f'{prefix}model_v2.pt', 'size': 524288000, 'modified': time.time() - 3600},
            {'key': f'{prefix}dataset.json', 'size': 10485760, 'modified': time.time() - 7200},
        ]
        
        return [obj for obj in objects if obj['key'].startswith(prefix)]
    
    def _get_storage_location(self, backend: str, key: str) -> str:
        """Get storage location URL."""
        if backend == 'local':
            return f"file://{self.backends[backend]['base_path']}/{key}"
        elif backend == 's3':
            return f"s3://{self.backends[backend]['bucket']}/{key}"
        elif backend == 'gcs':
            return f"gs://{self.backends[backend]['bucket']}/{key}"
        else:
            return f"{backend}://{key}"
    
    def get_storage_stats(self) -> Dict[str, Any]:
        """Get storage statistics."""
        stats = {}
        
        for backend_name, backend_config in self.backends.items():
            stats[backend_name] = {
                'type': backend_config['type'],
                'available': backend_config['available'],
                'objects_count': len(self.list_objects(backend=backend_name)),
                'total_size': sum(
                    obj['size'] for obj in self.list_objects(backend=backend_name)
                )
            }
        
        return stats


# Initialize storage service
storage_service = StorageServiceIntegration()

print("Storage Service Integration:")
print("="*50)

# Store different types of data
storage_operations = [
    {
        'key': 'models/deberta_v3_final.pt',
        'data': {'model_weights': 'simulated_weights'},
        'backend': 'local',
        'metadata': {'accuracy': 0.95, 'version': '1.0'}
    },
    {
        'key': 'datasets/ag_news_processed.json',
        'data': {'dataset': 'processed_data'},
        'backend': 'local',
        'metadata': {'samples': 120000, 'format': 'json'}
    },
    {
        'key': 'checkpoints/epoch_5.pt',
        'data': {'checkpoint': 'training_state'},
        'backend': 's3',  # Will fallback to local
        'metadata': {'epoch': 5, 'loss': 0.234}
    }
]

print("Storing objects:")
for op in storage_operations:
    result = storage_service.store(
        op['key'],
        op['data'],
        op['backend'],
        op['metadata']
    )
    print(f"  {result['key']}: {result['location']}")

# List stored objects
print("\nListing objects:")
for prefix in ['models/', 'datasets/', 'checkpoints/']:
    objects = storage_service.list_objects(prefix)
    print(f"\n{prefix}")
    for obj in objects:
        size_mb = obj['size'] / (1024 * 1024)
        print(f"  {obj['key']}: {size_mb:.1f} MB")

# Storage statistics
stats = storage_service.get_storage_stats()
print("\nStorage Statistics:")
for backend, backend_stats in stats.items():
    print(f"\n{backend}:")
    print(f"  Type: {backend_stats['type']}")
    print(f"  Available: {backend_stats['available']}")
    print(f"  Objects: {backend_stats['objects_count']}")
    print(f"  Total Size: {backend_stats['total_size'] / (1024**3):.2f} GB")

## 7. Monitoring and Alerting

In [ ]:
class MonitoringService:
    """
    Comprehensive monitoring and alerting service.
    
    Following monitoring practices from:
        Google SRE Book - "Monitoring Distributed Systems"
    """
    
    def __init__(self):
        self.metrics = {}
        self.alerts = []
        self.thresholds = self._default_thresholds()
        self._initialize_metrics()
    
    def _initialize_metrics(self):
        """Initialize metric collectors."""
        self.metrics['service_health'] = {}
        self.metrics['performance'] = {}
        self.metrics['errors'] = {}
        self.metrics['business'] = {}
    
    def _default_thresholds(self) -> Dict[str, Dict[str, float]]:
        """Define default alert thresholds."""
        return {
            'latency': {'warning': 500, 'critical': 1000},  # ms
            'error_rate': {'warning': 0.01, 'critical': 0.05},
            'cpu_usage': {'warning': 70, 'critical': 90},  # %
            'memory_usage': {'warning': 80, 'critical': 95},  # %
            'queue_depth': {'warning': 100, 'critical': 500}
        }
    
    def record_metric(
        self,
        category: str,
        name: str,
        value: float,
        tags: Optional[Dict[str, str]] = None
    ):
        """Record a metric value."""
        if category not in self.metrics:
            self.metrics[category] = {}
        
        if name not in self.metrics[category]:
            self.metrics[category][name] = []
        
        metric_point = {
            'value': value,
            'timestamp': time.time(),
            'tags': tags or {}
        }
        
        self.metrics[category][name].append(metric_point)
        
        # Check thresholds
        self._check_thresholds(name, value)
    
    def _check_thresholds(self, metric_name: str, value: float):
        """Check if metric exceeds thresholds."""
        if metric_name in self.thresholds:
            thresholds = self.thresholds[metric_name]
            
            if value >= thresholds['critical']:
                self._create_alert(
                    'CRITICAL',
                    metric_name,
                    value,
                    thresholds['critical']
                )
            elif value >= thresholds['warning']:
                self._create_alert(
                    'WARNING',
                    metric_name,
                    value,
                    thresholds['warning']
                )
    
    def _create_alert(self, severity: str, metric: str, value: float, threshold: float):
        """Create an alert."""
        alert = {
            'id': f"alert_{int(time.time() * 1000)}",
            'severity': severity,
            'metric': metric,
            'value': value,
            'threshold': threshold,
            'timestamp': time.time(),
            'status': 'active'
        }
        
        self.alerts.append(alert)
        logger.warning(
            f"{severity} Alert: {metric} = {value:.2f} (threshold: {threshold})"
        )
    
    def get_metrics_summary(self) -> Dict[str, Any]:
        """Get summary of all metrics."""
        summary = {}
        
        for category, metrics in self.metrics.items():
            summary[category] = {}
            
            for metric_name, points in metrics.items():
                if points:
                    values = [p['value'] for p in points[-100:]]  # Last 100 points
                    summary[category][metric_name] = {
                        'current': values[-1] if values else 0,
                        'avg': np.mean(values) if values else 0,
                        'min': np.min(values) if values else 0,
                        'max': np.max(values) if values else 0,
                        'p95': np.percentile(values, 95) if values else 0
                    }
        
        return summary
    
    def get_active_alerts(self) -> List[Dict[str, Any]]:
        """Get active alerts."""
        return [alert for alert in self.alerts if alert['status'] == 'active']


# Initialize monitoring service
monitoring = MonitoringService()

print("Monitoring Service:")
print("="*50)

# Simulate metric collection
print("Recording metrics:")

# Service metrics
service_metrics = [
    ('performance', 'latency', 250, {'service': 'prediction'}),
    ('performance', 'latency', 750, {'service': 'training'}),  # Will trigger warning
    ('performance', 'throughput', 150, {'service': 'prediction'}),
    ('errors', 'error_rate', 0.008, {'service': 'api'}),
    ('errors', 'error_rate', 0.06, {'service': 'storage'}),  # Will trigger critical
    ('service_health', 'cpu_usage', 65, {'node': 'worker-1'}),
    ('service_health', 'memory_usage', 72, {'node': 'worker-1'}),
    ('business', 'predictions_per_minute', 450, {}),
    ('business', 'models_trained', 5, {})
]

for category, name, value, tags in service_metrics:
    monitoring.record_metric(category, name, value, tags)
    print(f"  {category}.{name}: {value}")

# Get metrics summary
summary = monitoring.get_metrics_summary()

print("\nMetrics Summary:")
for category, metrics in summary.items():
    print(f"\n{category}:")
    for metric_name, stats in metrics.items():
        print(f"  {metric_name}:")
        print(f"    Current: {stats['current']:.2f}")
        print(f"    Average: {stats['avg']:.2f}")
        print(f"    P95: {stats['p95']:.2f}")

# Display active alerts
active_alerts = monitoring.get_active_alerts()

print("\nActive Alerts:")
if active_alerts:
    for alert in active_alerts:
        print(f"  [{alert['severity']}] {alert['metric']}: {alert['value']:.2f} > {alert['threshold']}")
else:
    print("  No active alerts")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Metric categories
categories = list(summary.keys())
metric_counts = [len(metrics) for metrics in summary.values()]

axes[0].bar(categories, metric_counts)
axes[0].set_xlabel('Category')
axes[0].set_ylabel('Number of Metrics')
axes[0].set_title('Metrics by Category')
axes[0].tick_params(axis='x', rotation=45)

# Alert severity distribution
severities = ['WARNING', 'CRITICAL']
severity_counts = [
    sum(1 for a in active_alerts if a['severity'] == s)
    for s in severities
]

axes[1].bar(severities, severity_counts, color=['orange', 'red'])
axes[1].set_xlabel('Severity')
axes[1].set_ylabel('Count')
axes[1].set_title('Active Alerts by Severity')

plt.tight_layout()
plt.show()

## 8. Notification Service

In [ ]:
class NotificationServiceIntegration:
    """
    Multi-channel notification service.
    
    Following notification patterns from:
        Martin Fowler - "Event Notification Pattern"
    """
    
    def __init__(self):
        self.channels = {}
        self.subscriptions = {}
        self.notification_history = []
        self._initialize_channels()
    
    def _initialize_channels(self):
        """Initialize notification channels."""
        self.channels['email'] = {
            'type': 'email',
            'enabled': True,
            'config': {'smtp_server': 'localhost', 'port': 587}
        }
        
        self.channels['slack'] = {
            'type': 'slack',
            'enabled': True,
            'config': {'webhook_url': 'https://hooks.slack.com/...'}
        }
        
        self.channels['webhook'] = {
            'type': 'webhook',
            'enabled': True,
            'config': {'endpoints': []}
        }
        
        self.channels['sms'] = {
            'type': 'sms',
            'enabled': False,
            'config': {'provider': 'twilio'}
        }
    
    def subscribe(
        self,
        event_type: str,
        channel: str,
        recipient: str,
        filters: Optional[Dict[str, Any]] = None
    ) -> str:
        """Subscribe to notifications."""
        subscription_id = f"sub_{int(time.time() * 1000)}"
        
        subscription = {
            'id': subscription_id,
            'event_type': event_type,
            'channel': channel,
            'recipient': recipient,
            'filters': filters or {},
            'created_at': time.time(),
            'active': True
        }
        
        if event_type not in self.subscriptions:
            self.subscriptions[event_type] = []
        
        self.subscriptions[event_type].append(subscription)
        
        return subscription_id
    
    def notify(
        self,
        event_type: str,
        title: str,
        message: str,
        severity: str = 'info',
        data: Optional[Dict[str, Any]] = None
    ) -> List[str]:
        """Send notifications for an event."""
        notification_ids = []
        
        # Get relevant subscriptions
        subscriptions = self.subscriptions.get(event_type, [])
        
        for subscription in subscriptions:
            if not subscription['active']:
                continue
            
            # Apply filters
            if not self._apply_filters(subscription['filters'], data):
                continue
            
            # Send notification
            notification_id = self._send_notification(
                subscription['channel'],
                subscription['recipient'],
                title,
                message,
                severity,
                data
            )
            
            notification_ids.append(notification_id)
        
        return notification_ids
    
    def _send_notification(
        self,
        channel: str,
        recipient: str,
        title: str,
        message: str,
        severity: str,
        data: Optional[Dict[str, Any]]
    ) -> str:
        """Send notification through specified channel."""
        notification_id = f"notif_{int(time.time() * 1000)}"
        
        notification = {
            'id': notification_id,
            'channel': channel,
            'recipient': recipient,
            'title': title,
            'message': message,
            'severity': severity,
            'data': data,
            'timestamp': time.time(),
            'status': 'sent'
        }
        
        self.notification_history.append(notification)
        
        # Simulate sending
        logger.info(f"Notification sent via {channel} to {recipient}")
        
        return notification_id
    
    def _apply_filters(
        self,
        filters: Dict[str, Any],
        data: Optional[Dict[str, Any]]
    ) -> bool:
        """Apply subscription filters."""
        if not filters:
            return True
        
        if not data:
            return False
        
        for key, value in filters.items():
            if key not in data or data[key] != value:
                return False
        
        return True
    
    def get_notification_history(
        self,
        limit: int = 10
    ) -> List[Dict[str, Any]]:
        """Get notification history."""
        return self.notification_history[-limit:]


# Initialize notification service
notification_service = NotificationServiceIntegration()

print("Notification Service:")
print("="*50)

# Setup subscriptions
subscriptions = [
    ('model_training_complete', 'email', 'admin@example.com', None),
    ('model_training_complete', 'slack', '#ml-team', None),
    ('error_alert', 'email', 'oncall@example.com', {'severity': 'critical'}),
    ('error_alert', 'slack', '#alerts', None),
    ('prediction_batch_complete', 'webhook', 'https://api.example.com/webhook', None)
]

print("Creating subscriptions:")
for event, channel, recipient, filters in subscriptions:
    sub_id = notification_service.subscribe(event, channel, recipient, filters)
    print(f"  {event} -> {channel}: {recipient}")

# Send notifications
print("\nSending notifications:")

# Model training complete
notif_ids = notification_service.notify(
    'model_training_complete',
    'Training Complete',
    'DeBERTa-v3 training completed successfully',
    'success',
    {'model': 'deberta-v3', 'accuracy': 0.95, 'duration': '2h 15m'}
)
print(f"  Sent {len(notif_ids)} notifications for training completion")

# Error alert
notif_ids = notification_service.notify(
    'error_alert',
    'Critical Error',
    'Storage service experiencing high error rate',
    'critical',
    {'service': 'storage', 'error_rate': 0.06, 'severity': 'critical'}
)
print(f"  Sent {len(notif_ids)} notifications for error alert")

# Get notification history
history = notification_service.get_notification_history(limit=5)

print("\nNotification History:")
for notif in history:
    print(f"\n  [{notif['severity'].upper()}] {notif['title']}")
    print(f"    Channel: {notif['channel']}")
    print(f"    Recipient: {notif['recipient']}")
    print(f"    Status: {notif['status']}")

## 9. Service Pipeline Integration

In [ ]:
class ServicePipelineIntegration:
    """
    End-to-end service pipeline integration.
    
    Following pipeline patterns from:
        Fowler (2014): "Microservices"
    """
    
    def __init__(
        self,
        service_discovery: ServiceDiscovery,
        orchestrator: ServiceOrchestrator,
        cache_service: CachingService,
        monitoring: MonitoringService
    ):
        self.service_discovery = service_discovery
        self.orchestrator = orchestrator
        self.cache_service = cache_service
        self.monitoring = monitoring
        self.pipelines = {}
    
    def create_pipeline(
        self,
        name: str,
        stages: List[Dict[str, Any]]
    ) -> str:
        """Create an integrated service pipeline."""
        pipeline_id = f"pipeline_{name}_{int(time.time())}"
        
        pipeline = {
            'id': pipeline_id,
            'name': name,
            'stages': stages,
            'status': 'created',
            'created_at': time.time()
        }
        
        self.pipelines[pipeline_id] = pipeline
        return pipeline_id
    
    async def execute_pipeline(
        self,
        pipeline_id: str,
        input_data: Dict[str, Any]
    ) -> Dict[str, Any]:
        """Execute integrated pipeline."""
        pipeline = self.pipelines.get(pipeline_id)
        if not pipeline:
            raise ValueError(f"Pipeline not found: {pipeline_id}")
        
        pipeline['status'] = 'running'
        stage_results = {}
        current_data = input_data
        
        for stage_idx, stage in enumerate(pipeline['stages']):
            stage_name = stage['name']
            
            # Check cache
            cache_key = f"{pipeline_id}_{stage_name}_{hash(str(current_data))}"
            cached_result = self.cache_service.get(cache_key)
            
            if cached_result:
                self.monitoring.record_metric(
                    'pipeline', 'cache_hits', 1,
                    {'pipeline': pipeline['name'], 'stage': stage_name}
                )
                stage_result = cached_result
            else:
                # Execute stage
                stage_result = await self._execute_stage(
                    stage, current_data
                )
                
                # Cache result
                self.cache_service.set(cache_key, stage_result, ttl=3600)
            
            stage_results[stage_name] = stage_result
            current_data = stage_result.get('output', current_data)
        
        pipeline['status'] = 'completed'
        pipeline['results'] = stage_results
        
        return pipeline
    
    async def _execute_stage(
        self,
        stage: Dict[str, Any],
        input_data: Dict[str, Any]
    ) -> Dict[str, Any]:
        """Execute a pipeline stage."""
        # Simulate stage execution
        await asyncio.sleep(0.2)
        
        return {
            'stage': stage['name'],
            'input': input_data,
            'output': {'processed': True, 'data': f"Processed {stage['name']}"},
            'metrics': {'latency': 200, 'items_processed': 100}
        }


# Create integrated pipeline
pipeline_integration = ServicePipelineIntegration(
    service_discovery,
    orchestrator,
    cache_service,
    monitoring
)

# Define classification pipeline
classification_pipeline_stages = [
    {'name': 'data_ingestion', 'service': 'data-service'},
    {'name': 'preprocessing', 'service': 'data-service'},
    {'name': 'feature_extraction', 'service': 'prediction-service'},
    {'name': 'model_inference', 'service': 'prediction-service'},
    {'name': 'postprocessing', 'service': 'prediction-service'},
    {'name': 'result_storage', 'service': 'storage-service'}
]

print("Service Pipeline Integration:")
print("="*50)

# Create pipeline
pipeline_id = pipeline_integration.create_pipeline(
    'classification_pipeline',
    classification_pipeline_stages
)

print(f"Created pipeline: {pipeline_id}")

# Execute pipeline
async def run_pipeline():
    input_data = {
        'texts': ['Sample news article'],
        'model_id': 'deberta-v3'
    }
    
    result = await pipeline_integration.execute_pipeline(
        pipeline_id,
        input_data
    )
    
    return result

# Run pipeline
pipeline_result = asyncio.run(run_pipeline())

print(f"\nPipeline Status: {pipeline_result['status']}")
print("\nStage Results:")

for stage_name, result in pipeline_result['results'].items():
    print(f"\n  {stage_name}:")
    print(f"    Output: {result['output']}")
    if 'metrics' in result:
        print(f"    Metrics: {result['metrics']}")

## 10. Service Health Dashboard

In [ ]:
# Create comprehensive service health dashboard
print("Service Health Dashboard:")
print("="*70)

# Service status
print("\n[SERVICE STATUS]")
print("-"*70)
services = service_discovery.list_services()
for service in services:
    status_icon = "[HEALTHY]" if service.status == 'healthy' else "[UNHEALTHY]"
    print(f"{status_icon} {service.name:20} {service.status:10} {service.host}:{service.port}")

# Queue status
print("\n[MESSAGE QUEUES]")
print("-"*70)
for queue_name, queue_data in mq_service.queues.items():
    message_count = len(queue_data['messages'])
    print(f"  {queue_name:25} Messages: {message_count:3}")

# Cache statistics
cache_stats = cache_service.get_stats()
print("\n[CACHE PERFORMANCE]")
print("-"*70)
print(f"  Hit Rate:     {cache_stats['hit_rate']:.1%}")
print(f"  Total Hits:   {cache_stats['hits']}")
print(f"  Total Misses: {cache_stats['misses']}")
print(f"  Evictions:    {cache_stats['evictions']}")

# Monitoring metrics
metrics_summary = monitoring.get_metrics_summary()
print("\n[SYSTEM METRICS]")
print("-"*70)
if 'performance' in metrics_summary:
    for metric, stats in metrics_summary['performance'].items():
        print(f"  {metric:20} Current: {stats['current']:8.2f}  P95: {stats['p95']:8.2f}")

# Active alerts
active_alerts = monitoring.get_active_alerts()
print("\n[ACTIVE ALERTS]")
print("-"*70)
if active_alerts:
    for alert in active_alerts:
        severity_indicator = "[CRITICAL]" if alert['severity'] == 'CRITICAL' else "[WARNING]"
        print(f"{severity_indicator} {alert['metric']:15} = {alert['value']:.2f}")
else:
    print("  No active alerts")

# Storage usage
storage_stats = storage_service.get_storage_stats()
print("\n[STORAGE USAGE]")
print("-"*70)
for backend, stats in storage_stats.items():
    if stats['available']:
        size_gb = stats['total_size'] / (1024**3)
        print(f"  {backend:10} Objects: {stats['objects_count']:5}  Size: {size_gb:.2f} GB")

print("\n" + "="*70)

## 11. Conclusions and Next Steps

### Service Integration Summary

This tutorial demonstrated comprehensive service integration patterns:

1. **Service Discovery**: Registry and health checking
2. **Orchestration**: Workflow management and execution
3. **Message Queuing**: Asynchronous processing with priorities
4. **Caching**: Multi-level caching with TTL and eviction
5. **Storage**: Unified interface for multiple backends
6. **Monitoring**: Metrics collection and alerting
7. **Notifications**: Multi-channel event notifications
8. **Pipeline Integration**: End-to-end service pipelines

### Key Takeaways

1. **Service Decoupling**: Independent services with clear interfaces
2. **Resilience Patterns**: Health checks, retries, circuit breakers
3. **Performance Optimization**: Caching, batching, async processing
4. **Observability**: Comprehensive monitoring and alerting
5. **Scalability**: Distributed architecture with queue-based processing

### Next Steps

1. **Advanced Orchestration**:
   - Implement Saga pattern for distributed transactions
   - Add workflow versioning and rollback
   - Implement dynamic service mesh

2. **Performance Enhancement**:
   - Add distributed tracing
   - Implement adaptive caching strategies
   - Use connection pooling

3. **Reliability Improvements**:
   - Implement chaos engineering tests
   - Add automated failover
   - Create disaster recovery procedures

4. **Production Deployment**:
   - Container orchestration with Kubernetes
   - Service mesh with Istio
   - Continuous deployment pipelines

### References

For deeper understanding, consult:
- Service documentation: `docs/developer_guide/service_development.md`
- Architecture patterns: `docs/architecture/patterns/`
- Operations guide: `docs/operations/runbooks/`
- Monitoring setup: `monitoring/dashboards/`